In [23]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd
import os

# Connection parameters
hostName = "https://c2fa4efab8ca4cb28b9d08be859caa2c.i.tgcloud.io/"
secret = "320tcb2tn2d1u8oo7iupkcl11oi0k1dj"

conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="MyGraph")
conn.getToken(secret)

('204e9hfhv3eg2dps5kr9p10uaet4um15', 1676766082, '2023-02-19 00:21:22')

### Define and Publish Graph Schema

Nodes = ETH Users/wallets: ID, label <br />
Directed Edges = Transaction flow: From_ID, To_ID, ETH_amount, timestamp

In [24]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX Wallet (PRIMARY_ID id STRING, label STRING) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE sent_eth (from Wallet, to Wallet, amount INT, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_eth"
  CREATE DIRECTED EDGE received_eth (from Wallet, to Wallet, amount INT, receive_date DATETIME) WITH REVERSE_EDGE="reverse_received_eth"
''')
print(results)

Successfully created vertex types: [Wallet].
Successfully created edge types: [sent_eth].
Successfully created reverse edge types: [reverse_sent_eth].
Successfully created edge types: [received_eth].
Successfully created reverse edge types: [reverse_received_eth].


### Create the Ethereum Transaction Graph

In [26]:
results = conn.gsql('''
  CREATE GRAPH MyGraph2(Wallet, sent_eth, reverse_sent_eth, received_eth, reverse_received_eth)
''')
print(results)

The graph MyGraph2 is created.


In [28]:
conn.graphname="ETH_Transaction_Graph"
#secret = conn.createSecret()
# authToken = conn.getToken(secret)
# authToken = authToken[0]
# print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="ETH_Transaction_Graph")

def pprint(string):
  print(json.dumps(string, indent=2))

### Create Loading Jobs

#### Wallets

In [31]:
results = conn.gsql('''
  USE GRAPH MyGraph2
  BEGIN
  CREATE LOADING JOB load_wallets FOR GRAPH MyGraph2 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Wallet VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')
print(results)

Using graph 'MyGraph2'
Successfully created loading jobs: [load_wallets].


#### Transactions

In [32]:
results = conn.gsql('''
  USE GRAPH MyGraph2
  BEGIN
  CREATE LOADING JOB load_transactions FOR GRAPH MyGraph2 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE sent_eth VALUES($1, $0, $2, $3) USING SEPARATOR=",", HEADER="false", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE received_eth VALUES($0, $1, $2, $3) USING SEPARATOR=",", HEADER="false", EOL="\\n", QUOTE="double";
  }
  END''')
print(results)

Using graph 'MyGraph2'
Successfully created loading jobs: [load_transactions].


### Load Data

In [34]:
os.chdir('/Users/stephengelinas/DSC180A-Q2-Project/data')
# nodes = pd.read_csv(r'nodes.txt', header=None, sep=' ', names=['id','label'])
# edges = pd.read_csv(r'edges.txt', header=None, names=['from', 'to', 'amount', 'timestamp'])
# nodes.to_csv('nodes.csv',index=False)
# edges.to_csv('edges.csv',index=False)

In [35]:
# Load the nodes file with the 'load_wallets' job
posts_file = 'nodes.csv'
results = conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_wallets')
print(json.dumps(results, indent=2))

TigerGraphException: ("The graph name 'ETH_Transaction_Graph' parsed from the url = '/ddl/ETH_Transaction_Graph' is not found, please provide a valid graph name.", 'REST-1004')

In [ ]:
# Load the edges file with the 'load_transactions' job
posts_file = 'edges.csv'
results = conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_transactions')
print(json.dumps(results, indent=2))

### Exploring the Graph

In [ ]:
# TODO - ETHAN

# write GSQL queries to get some summary statistics on edges/nodes
# use this as reference: https://colab.research.google.com/drive/1JhYcnGVWT51KswcXZzyPzKqCoPP5htcC?usp=sharing#scrollTo=Zerob6pDgmq2 

### Delete Graph

DANGER!!! DON'T RUN THIS!!!!

In [13]:
# conn.gsql('''
# USE GLOBAL
# DROP GRAPH MyGraph
# ''')

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nTry to abort all loading jobs on graph MyGraph, it may take a while ...\n[ABORT_SUCCESS] No active Loading Job to abort.\nResetting GPE...\nSuccessfully reset GPE and GSE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 0.002 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.275 seconds\nEverything is dropped.'